In [ ]:
# Load a triangle mesh
from lapy import TriaMesh, TriaIO

In [ ]:
T = TriaIO.import_off("../data/square-mesh.off")
type(T).__name__

In [ ]:
# import plotting functions
import plotly
plotly.offline.init_notebook_mode(connected=True)
from lapy.Plot import plot_tria_mesh

In [ ]:
# plot the mesh (here with a triangle function overlay of tria quality)
q = T.tria_qualities()
# we commented all plot calls, so that the notebook size remains small, uncomment and look at it
#plot_tria_mesh(T,plot_edges=True, tfunc=q)

In [ ]:
# compute first eigenfunction
from lapy import Solver
fem = Solver(T,lump=True)
eval, evec = fem.eigs()
vfunc = evec[:,1]

# also get A,B (lumped), and inverse of B (easy as it is diagonal due to lumping)
A, B = fem.stiffness, fem.mass
Bi = B.copy()
Bi.data **= -1

In [ ]:
# note the mass matrix B can be used as inner product like x B x'. 
# for a lumped matrix (diagonal) it has contains the vertex areas at the diagonal. 
# in any case, the sum of all elements is the total mesh area (1 in the case of the unit square)
B.sum()

In [ ]:
# when applying the Laplace to an eigenfunction you get a scaled version of that function 
#plot_tria_mesh(T,Bi*(A*vfunc),plot_edges=True)

In [ ]:
# so it is the same as the corresponding eigenvalue times the eigenfunction
#plot_tria_mesh(T,eval[1]*vfunc,plot_edges=True)

In [ ]:
# individually applying first gradient and then - divergence (and inverse of B) gives the same function
# inverse of B is used to get back from the integrated divergence to the function 
from lapy.DiffGeo import compute_gradient
from lapy.DiffGeo import compute_divergence
grad = compute_gradient(T,vfunc)
divx = -compute_divergence(T,grad)
#plot_tria_mesh(T,Bi*divx,plot_edges=True)

In [ ]:
# now we compute heat diffusion from boundary vertices with default m
from lapy import Heat
bvert = T.boundary_loops()
u = Heat.diffusion(T,bvert,m=1)

In [ ]:
# show some of the level sets (not evenly spaced, steeper close to the boundary)
#plot_tria_mesh(T,u,plot_edges=True,plot_levels=True)

In [ ]:
import numpy as np

# compute gradient of heat diffusion
tfunc = compute_gradient(T,u)

# normalize gradient
X = -tfunc / np.sqrt((tfunc**2).sum(1))[:,np.newaxis]
X = np.nan_to_num(X)

In [ ]:
# compute divergence of normalized gradient
divx = compute_divergence(T,X)

In [ ]:
# compute distance
from scipy.sparse.linalg import splu
useCholmod = True
try:
    from sksparse.cholmod import cholesky
except ImportError:
    useCholmod = False

fem = Solver(T,lump=True)
A, B = fem.stiffness, fem.mass

H=-A
b0=divx
        
# solve H x = b0
# we don't need the B matrix here, as divx is the intgrated divergence 
print("Matrix Format now: "+H.getformat())
if useCholmod:
    print("Solver: cholesky decomp - performance optimal ...")
    chol = cholesky(H)
    x = chol(b0)
else:
    print("Solver: spsolve (LU decomp) - performance not optimal ...")
    lu = splu(H)
    x = lu.solve(b0)

# remove shift
x = x-min(x)

In [ ]:
#plot_tria_mesh(T,x,plot_edges=True,plot_levels=True)

In [ ]:
# max distance (smoothed)
(max(x), np.sqrt(2)/2)

In [ ]:
# we can also use the standard Poisson solver on B inverse time divx
vf = fem.poisson(-Bi*divx)
vf = vf - min(vf)
#plot_tria_mesh(T,vf,plot_edges=True,plot_levels=True)

In [ ]:
# should be the same as what we had earlier
max(abs(vf-x))

In [ ]:
# or just call this, which does all the work for us
from lapy.DiffGeo import compute_geodesic_f
gf = compute_geodesic_f(T,u)
#plot_tria_mesh(T,gf,plot_edges=True,plot_levels=True)

In [ ]:
# again should be the same
max(abs(gf-x))

In [ ]:
# testing if we can rotate the function
from lapy.DiffGeo import compute_rotated_f
gf = compute_rotated_f(T,vf)
#plot_tria_mesh(T,gf,plot_edges=True,plot_levels=True)

In [ ]:
# To demonstrate the mean curvature mapping to a sphere
# we need to have a closed mesh
from lapy.DiffGeo import tria_mean_curvature_flow
from lapy.Plot import plot_tria_mesh
# load your mesh here and uncomment. The mesh should have not too many flat regions (not a cube)
#T = TriaIO.import_off("../data/???")
#T2 = tria_mean_curvature_flow(T)
#plot_tria_mesh(T2,plot_edges=True,plot_levels=True)

